# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments              Customer  \
0  Oct  4 2022  3:52PM  247899        19  ADV80006215   AdvaGen Pharma, Ltd   
1  Oct  4 2022  3:52PM  247899        19  ADV80006216   AdvaGen Pharma, Ltd   
2  Oct  4 2022  3:52PM  247899        19  ADV80006217   AdvaGen Pharma, Ltd   
3  Oct  4 2022  3:52PM  247899        19  ADV80006218   AdvaGen Pharma, Ltd   
4  Oct  4 2022  3:52PM  247899        19  ADV80006219   AdvaGen Pharma, Ltd   
5  Oct  4 2022  3:52PM  247899        19  ADV80006220   AdvaGen Pharma, Ltd   
6  Oct  4 2022  3:52PM  247899        19  ADV80006221   AdvaGen Pharma, Ltd   
7  Oct  4 2022  3:46PM  247897        15   PNC387526A  Person & Covey, Inc.   
8  Oct  4 2022  3:46PM  247897        15   PNC387616A  Person & Covey, Inc.   
9  Oct  4 2022  3:46PM  247897        15   PNC387621A  Person & Covey, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
26  247896      Executing          8
27  247896       Released          6
28  247897      Executing          3
29  247897       Released          6
30  247899       Released          7

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
247892                NaN        NaN       1.0
247895                NaN        NaN       1.0
247896                NaN        8.0       6.0
247897                NaN        3.0       6.0
247899                NaN        NaN       7.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247796                0.0        7.0       1.0
247811                3.0       10.0      17.0
247839                0.0        1.0       0.0
247853                0.0        0.0       6.0
247864                0.0        5.0      14.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
247796                  0          7         1
247811                  3         10        17
247839                  0          1         0
247853                  0          0         6
247864                  0          5        14

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               247796          0          7         1
1               247811          3         10        17
2               247839          0          1         0
3               247853          0          0         6
4               247864          0          5        14

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               247796                   7        1
1               247811         3        10       17
2               247839                   1         
3               247853                            6
4               247864                   5       14

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0   Oct  4 2022  3:52PM  247899        19            AdvaGen Pharma, Ltd
7   Oct  4 2022  3:46PM  247897        15           Person & Covey, Inc.
16  Oct  4 2022  3:40PM  247896        15           Person & Covey, Inc.
30  Oct  4 2022  3:20PM  247895        10               Eywa Pharma Inc.
31  Oct  4 2022  3:09PM  247892        10               Eywa Pharma Inc.
32  Oct  4 2022  2:59PM  247883        10              ISDIN Corporation
61  Oct  4 2022  2:59PM  247887        10              ISDIN Corporation
71  Oct  4 2022  2:58PM  247889        16  Sartorius Stedim Filters Inc.
76  Oct  4 2022  2:57PM  247888        16  Sartorius Stedim Filters Inc.
82  Oct  4 2022  2:57PM  247884        18             Innova Softgel LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Oct  4 2022  3:52PM  247899        19            AdvaGen Pharma, Ltd   
1  Oct  4 2022  3:46PM  247897        15           Person & Covey, Inc.   
2  Oct  4 2022  3:40PM  247896        15           Person & Covey, Inc.   
3  Oct  4 2022  3:20PM  247895        10               Eywa Pharma Inc.   
4  Oct  4 2022  3:09PM  247892        10               Eywa Pharma Inc.   
5  Oct  4 2022  2:59PM  247883        10              ISDIN Corporation   
6  Oct  4 2022  2:59PM  247887        10              ISDIN Corporation   
7  Oct  4 2022  2:58PM  247889        16  Sartorius Stedim Filters Inc.   
8  Oct  4 2022  2:57PM  247888        16  Sartorius Stedim Filters Inc.   
9  Oct  4 2022  2:57PM  247884        18             Innova Softgel LLC   

  Completed Executing Released  
0                            7  
1                   3        6  
2                   8        6  
3                            1  
4                            1  
5                           29  
6                   3        7  
7                            5  
8                            6  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Oct  4 2022  3:52PM  247899        19            AdvaGen Pharma, Ltd   
1  Oct  4 2022  3:46PM  247897        15           Person & Covey, Inc.   
2  Oct  4 2022  3:40PM  247896        15           Person & Covey, Inc.   
3  Oct  4 2022  3:20PM  247895        10               Eywa Pharma Inc.   
4  Oct  4 2022  3:09PM  247892        10               Eywa Pharma Inc.   
5  Oct  4 2022  2:59PM  247883        10              ISDIN Corporation   
6  Oct  4 2022  2:59PM  247887        10              ISDIN Corporation   
7  Oct  4 2022  2:58PM  247889        16  Sartorius Stedim Filters Inc.   
8  Oct  4 2022  2:57PM  247888        16  Sartorius Stedim Filters Inc.   
9  Oct  4 2022  2:57PM  247884        18             Innova Softgel LLC   

  Released Executing Completed  
0        7                      
1        6         3            
2        6         8            
3        1                      
4        1                      
5       29                      
6        7         3            
7        5                      
8        6                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse              Customer Released  \
0  Oct  4 2022  3:52PM  247899        19   AdvaGen Pharma, Ltd        7   
1  Oct  4 2022  3:46PM  247897        15  Person & Covey, Inc.        6   
2  Oct  4 2022  3:40PM  247896        15  Person & Covey, Inc.        6   
3  Oct  4 2022  3:20PM  247895        10      Eywa Pharma Inc.        1   
4  Oct  4 2022  3:09PM  247892        10      Eywa Pharma Inc.        1   

  Executing Completed  
0                      
1         3            
2         8            
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Oct  4 2022  3:52PM  247899        19   AdvaGen Pharma, Ltd       7.0   
1  Oct  4 2022  3:46PM  247897        15  Person & Covey, Inc.       6.0   
2  Oct  4 2022  3:40PM  247896        15  Person & Covey, Inc.       6.0   
3  Oct  4 2022  3:20PM  247895        10      Eywa Pharma Inc.       1.0   
4  Oct  4 2022  3:09PM  247892        10      Eywa Pharma Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        3.0        NaN  
2        8.0        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse              Customer  Released  \
0  Oct  4 2022  3:52PM  247899        19   AdvaGen Pharma, Ltd       7.0   
1  Oct  4 2022  3:46PM  247897        15  Person & Covey, Inc.       6.0   
2  Oct  4 2022  3:40PM  247896        15  Person & Covey, Inc.       6.0   
3  Oct  4 2022  3:20PM  247895        10      Eywa Pharma Inc.       1.0   
4  Oct  4 2022  3:09PM  247892        10      Eywa Pharma Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        3.0        0.0  
2        8.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2230841      77.0       20.0        5.0
15          495793      12.0       11.0        0.0
16          991459      18.0        7.0        0.0
18          495723       1.0        1.0        0.0
19          743653      18.0       11.0        0.0
20          247875       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2230841      77.0       20.0        5.0
1        15   495793      12.0       11.0        0.0
2        16   991459      18.0        7.0        0.0
3        18   495723       1.0        1.0        0.0
4        19   743653      18.0       11.0        0.0
5        20   247875       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      77.0       20.0        5.0
1        15      12.0       11.0        0.0
2        16      18.0        7.0        0.0
3        18       1.0        1.0        0.0
4        19      18.0       11.0        0.0
5        20       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   77.0
1        15  Released   12.0
2        16  Released   18.0
3        18  Released    1.0
4        19  Released   18.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    16   18    19   20
Status                                     
Completed   5.0   0.0   0.0  0.0   0.0  0.0
Executing  20.0  11.0   7.0  1.0  11.0  0.0
Released   77.0  12.0  18.0  1.0  18.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    16   18    19   20
0          Completed   5.0   0.0   0.0  0.0   0.0  0.0
1          Executing  20.0  11.0   7.0  1.0  11.0  0.0
2           Released  77.0  12.0  18.0  1.0  18.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    16   18    19   20
0  Completed   5.0   0.0   0.0  0.0   0.0  0.0
1  Executing  20.0  11.0   7.0  1.0  11.0  0.0
2   Released  77.0  12.0  18.0  1.0  18.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()